In [1]:
import findspark
findspark.init("/home/gagandeep/spark/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
import os

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [8]:
pathToFile = "/media/gagandeep/2E92405C92402AA3/Work/Codes/PythonCodes/SparkLesson/DataFiles/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection/SMSSpamCollection"
data = spark.read.csv(pathToFile, inferSchema=True, header=False, sep='\t')

In [9]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [10]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [11]:
data.columns

['_c0', '_c1']

In [12]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [13]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import length

In [15]:
data = data.withColumn('length', length(data['text']))

In [16]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [17]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [18]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [31]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')


In [40]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [41]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [42]:
from pyspark.ml import Pipeline

In [43]:
pipeline = Pipeline(stages=[ham_spam_to_num, tokenizer, stop_remove, count_vec, idf,clean_up])

In [44]:
cleaned = pipeline.fit(data)


In [45]:
clean_data = cleaned.transform(data).select('label','features')
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [46]:
train, test = clean_data.randomSplit([0.7,0.3])

In [47]:
spam_detector = nb.fit(train)

In [48]:
test_res = spam_detector.transform(test)

In [49]:
test_res.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-1155.1059700659...|[1.0,3.3449208628...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-666.19765682410...|[1.0,5.8396932461...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-559.40480043221...|[1.0,1.2017267760...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-559.40480043221...|[1.0,1.2017267760...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-967.90478154712...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1005.6869018823...|[1.0,7.8420248012...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1470.5250150893...|[0.99999999999996...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1139.8702314684...|[2.57915677431715...|       1.0|
|  0.0|(13424,[0,1,146,1...|[-248.68090543735...|[0.90190892902948...|       0.0|
|  0.0|(13424,[0

In [55]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()

In [56]:
acc = acc_eval.evaluate(test_res)
acc

0.9273072258824983